# Automated ML

Importing all needed dependencies to complete the project.

In [1]:
import logging
import os
import json
import csv
import numpy as np
import pandas as pd
import pkg_resources
import joblib

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails
from azureml.core import Model, Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from sklearn.preprocessing import StandardScaler

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


## Dataset

#### Overview
This dataset comes from the Diabetes and Digestive and Kidney Disease National Institutes. 
The purpose of this dataset is to diagnose whether or not a patient is diabetic, on the basis of certain diagnostic measures in the dataset. 
The selection of these instances from a larger database was subject to several restrictions. 
All patients are women from the Indian heritage of Pima, at least 21 years old. Datasets can be found here:
https://www.kaggle.com/uciml/pima-indians-diabetes-database

#### Task
Here, we plan to predict the "Outcome" column based on the input features, either the patient has diabetes or not.


The dataset has 9 variables including:
- Pregnancies: Number pregnancy times (int).
- Glucose: Plasma glucose concentration level (int).
- BloodPressure: Diastolic blood pressure level in mm Hg(int).
- SkinThickness: skinfold thickness in mm(int).
- Insulin: two-hour serum insulin measured by mu U/ml(int).
- BMI: Body mass index (float).
- DiabetesPedigreeFunction: Diabetes pedigree function(float).
- Age: age in years 21 and above(int).
- Outcome: Target column 0 or 1, 0 = Not diabetes, 1 = diabetes(int).

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl-capstone'

experiment=Experiment(ws, experiment_name)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AXX5D7CH8 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [3]:
# Load the registered dataset from workspace
dataset = Dataset.get_by_name(ws, name='diabete')

# Convert the dataset to dataframe
df = dataset.to_pandas_dataframe()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
Pregnancies                 768 non-null int64
Glucose                     768 non-null int64
BloodPressure               768 non-null int64
SkinThickness               768 non-null int64
Insulin                     768 non-null int64
BMI                         768 non-null float64
DiabetesPedigreeFunction    768 non-null float64
Age                         768 non-null int64
Outcome                     768 non-null int64
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [4]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [5]:
#Display the first five records of the dataset
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [13]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [6]:
# Create CPU cluster
amlcompute_cluster_name = "notebook138912"

# Verify if cluster does not exist otherwise use the existing one
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS12_V2',
                                                           vm_priority = 'lowpriority', 
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)


Found existing cluster, use it.

Running


## AutoML Configuration

Overview of the automl settings and configuration used for this experiment:

- "experiment_timeout_minutes": set to 30 minutes. The experiment will timeout after that period to avoid wasting resources.
- "max_concurrent_iterations": is set to 4. The max number of concurrent iterations to be run in parallel at the same time.
- "primary_metric" : is set to 'accuracy', which is a sutible metric for classification problems.
- "n_cross_validations": is set to 5, therefore the training and validation sets will be divided into five equal sets.
- "iterations": the number of iterations for the experiment is set to 20. It's a reasonable number and would provide the intendable result for the given dataset.
- compute_target: set to the project cluster to run the experiment.
- task: set to 'classification' since our target to predict whether the patient has diabetes or not.
- training_data: the loaded dataset for the project.
- label_column_name: set to the result/target colunm in the dataset 'Outcome' (0 or 1).
- enable_early_stopping: is enabled to terminate the experiment if the accuracy score is not showing improvement over time.
- featurization = is set to 'auto', it's an indicator of whether implementing a featurization step to preprocess/clean the dataset automatically or not. In our case, the preprocessing was applied for the numerical columns which normally involve treating missing values, cluster distance, the weight of evidence...etc.
- debug_log: errors will be logged into 'automl_errors.log'.


In [7]:
# Automl settings 
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy',
    "n_cross_validations": 5,
    "iterations": 24
    
}

# Automl config 
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = 'classification',
                             training_data=dataset,
                             label_column_name='Outcome',
                             enable_early_stopping= True,
                             featurization = 'auto',
                             debug_log = 'automl_errors.log',
                             **automl_settings
                            )



In [8]:
# Submit experiment
remote_run = experiment.submit(automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on notebook138912 with default configuration
Running on remote compute: notebook138912
Parent Run ID: AutoML_01843660-d936-4a03-a31a-ece32909196e

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:      

## Run Details

The best model has resulted from the AutoML experiment from VotingEnsemble model. The Voting Ensemble model takes a majority vote of several algorithms which makes it surpass individual algorithms and minimize the bias. 

Use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [14]:
remote_run.wait_for_completion(show_output=True)



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_01843660-d936-4a03-a31a-ece32909196e',
 'target': 'notebook138912',
 'status': 'Completed',
 'startTimeUtc': '2021-02-14T21:30:05.346613Z',
 'endTimeUtc': '2021-02-14T21:45:24.245199Z',
 'properties': {'num_iterations': '24',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'notebook138912',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-capstone","subscription_id":"b968fb36-f06a-4c76-a15f-afab68ae7667","resource_group":"aml-quickstarts-138912","workspace_name":"quick-starts-ws-138912","region":"southcentralus","compute_target":"notebook138912","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":24,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":

In [15]:
remote_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl-capstone,AutoML_01843660-d936-4a03-a31a-ece32909196e,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation


## Best Model

The cell below shows the best model from the automl experiments and display all the properties of the model.



In [17]:
# find and save best automl model
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model.steps)

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


Run(Experiment: automl-capstone,
Id: AutoML_01843660-d936-4a03-a31a-ece32909196e_22,
Type: azureml.scriptrun,
Status: Completed)
[('datatransformer', DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                feature_sweeping_config=None, feature_sweeping_timeout=None,
                featurization_config=None, force_text_dnn=None,
                is_cross_validation=None, is_onnx_compatible=None, logger=None,
                observer=None, task=None, working_dir=None)), ('prefittedsoftvotingclassifier', PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('17',
                                           Pipeline(memory=None,
                                                    steps=[('maxabsscaler',
                                                            MaxAbsScaler(copy=True)),
                                                           ('logisticregression',
                                                      

In [18]:
print(fitted_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                               reg_alpha=0.5208333333333334,
                                                                                               reg_lambda=0.8333333333333334,
                                                                                               scale_pos_weight=1,
                          

In [19]:
best_run_metrics=remote_run.get_metrics()

In [20]:
# Show all metrics for our best run
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

experiment_status ['DatasetEvaluation', 'FeaturesGeneration', 'DatasetFeaturization', 'DatasetFeaturizationCompleted', 'DatasetCrossValidationSplit', 'ModelSelection']
experiment_status_description ['Gathering dataset statistics.', 'Generating features for the dataset.', 'Beginning to fit featurizers and featurize the dataset.', 'Completed fit featurizers and featurizing the dataset.', 'Generating individually featurized CV splits.', 'Beginning model selection.']
matthews_correlation 0.5100479946254547
precision_score_macro 0.7767529003830413
recall_score_micro 0.787810881928529
f1_score_macro 0.7461635802185229
weighted_accuracy 0.8288071848964478
log_loss 0.4805423157349399
accuracy 0.787810881928529
average_precision_score_weighted 0.8429799420241949
precision_score_weighted 0.7871495358175469
recall_score_macro 0.7351453929387143
AUC_macro 0.834385233677083
recall_score_weighted 0.787810881928529
average_precision_score_macro 0.8136057681069753
average_precision_score_micro 0.85391

In [22]:
BestModel = best_run.register_model(model_path='outputs/model.pkl', model_name='diabeteModel_automl',
                        tags={'Training context':'Auto ML'},
                        properties={'Accuracy': best_run_metrics['accuracy']})

print(BestModel)

Model(workspace=Workspace.create(name='quick-starts-ws-138912', subscription_id='b968fb36-f06a-4c76-a15f-afab68ae7667', resource_group='aml-quickstarts-138912'), name=diabeteModel_automl, id=diabeteModel_automl:1, version=1, tags={'Training context': 'Auto ML'}, properties={'Accuracy': '0.787810881928529'})


In [23]:
# SHow that our model saved
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

diabeteModel_automl version: 1
	 Training context : Auto ML
	 Accuracy : 0.787810881928529


capstoneModel_automl version: 1
	 Training context : Auto ML
	 Accuracy : 0.787810881928529




## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

In the cell below, we have registered the model, created an inference config and deployed the model as a web service.

In [36]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice.aci import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model

In [38]:
env = best_run.get_environment().save_to_directory(path='environments')

In [40]:
script_file= 'score.py'

best_run.download_file('outputs/scoring_file_v_1_0_0.py', script_file)

In [32]:
# Registring the best model
model = best_model.register_model(model_name='automl-best-model',model_path='outputs/model.pkl')


In [41]:
inference_config = InferenceConfig(entry_script='score.py',
 environment=environment)


In [42]:
service_name = 'automl-deploy'
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=deployment_config,
                       overwrite=True
                      )
service.wait_for_deployment(show_output=True)

scoring_uri = service.scoring_uri
print(scoring_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running............................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
http://f7511453-d747-4906-b13b-8de05ba86490.southcentralus.azurecontainer.io/score


In [43]:
# Enable app insights
service.update(enable_app_insights=True)

In the cell below, we have sent a request to the web service to test it.

In [44]:
import requests
import json

In [45]:


#two set of data to score, so we get two results back
data = {"data": [{"Pregnancies": 5, 
     "Glucose": 150, 
     "BloodPressure": 70, 
     "SkinThickness": 40, 
     "Insulin": 10, 
     "BMI": 36.5, 
     "DiabetesPedigreeFunction": 0.627, 
     "Age": 30},

    {"Pregnancies": 5, 
     "Glucose": 90, 
     "BloodPressure": 70, 
     "SkinThickness": 34, 
     "Insulin": 20, 
     "BMI": 26.5, 
     "DiabetesPedigreeFunction": 0.351, 
     "Age": 28},
      ]}
    
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
#headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())
print("Case 0: Not Diabetes, Case 1: Diabetes.")

{"result": [1, 0]}
Case 0: Not Diabetes, Case 1: Diabetes.


In [56]:
data = [{"Pregnancies": 12, 
     "Glucose": 190, 
     "BloodPressure": 120, 
     "SkinThickness": 60, 
     "Insulin": 700, 
     "BMI": 56.5, 
     "DiabetesPedigreeFunction": 1.5, 
     "Age": 70},

    {"Pregnancies": 6, 
     "Glucose": 40, 
     "BloodPressure": 70, 
     "SkinThickness": 24, 
     "Insulin": 0, 
     "BMI": 25.5, 
     "DiabetesPedigreeFunction": 0.351, 
     "Age": 22},
      ]

print(data)

[{'Pregnancies': 12, 'Glucose': 190, 'BloodPressure': 120, 'SkinThickness': 60, 'Insulin': 700, 'BMI': 56.5, 'DiabetesPedigreeFunction': 1.5, 'Age': 70}, {'Pregnancies': 6, 'Glucose': 40, 'BloodPressure': 70, 'SkinThickness': 24, 'Insulin': 0, 'BMI': 25.5, 'DiabetesPedigreeFunction': 0.351, 'Age': 22}]


In [57]:
# test using service instance
input_data = json.dumps({
    'data': data
})

output = service.run(input_data)
output

'{"result": [1, 0]}'

Print the logs of the web service and delete the service

In [58]:
logs = service.get_logs()
logs

'2021-02-14T22:39:43,339067300+00:00 - gunicorn/run \n2021-02-14T22:39:43,339106100+00:00 - iot-server/run \n2021-02-14T22:39:43,352195300+00:00 - rsyslog/run \n2021-02-14T22:39:43,399722400+00:00 - nginx/run \n/usr/sbin/nginx: /azureml-envs/azureml_09ff55f546b313bb1ab136a466214499/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_09ff55f546b313bb1ab136a466214499/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_09ff55f546b313bb1ab136a466214499/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_09ff55f546b313bb1ab136a466214499/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_09ff55f546b313bb1ab136a466214499/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)

In [26]:
service.delete()